<a href="https://colab.research.google.com/github/DOI03/DOI03/blob/main/CV_AL_test7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.   optimizer로 AdamP 사용
2.   lr 0.0001
3.   epoch 25
4.   get_cosine_schedule 스케줄러 사용



In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
!pip install wandb
!pip install adamp

In [5]:
import os
import time

import torch
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

import wandb

# AdamP 사용
from adamp import AdamP

from PIL import Image
#내가 추가
import re

from transformers import get_cosine_schedule_with_warmup

### wandb 설정 ###
wandb.init(project="CUB_Transfer_Learning", name="AdamP_with_get_cosine_schedule")

### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print(DEVICE)

cuda


In [6]:
### Custom Dataset ###
class CUB2011(Dataset):
  def __init__(self, transform, mode='train'):
    self.transform = transform
    self.mode = mode

    if self.mode == 'train':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets/train')
    elif self.mode == 'valid':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets/valid')
    elif self.mode == 'test':
      self.image_folder = os.listdir('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets/test')

  def __len__(self):
    return len(self.image_folder)

  def __getitem__(self, idx):
    img_path = self.image_folder[idx]
    img = Image.open(os.path.join('/content/gdrive/MyDrive/CUB_200_2011_repackage_class50/datasets', self.mode, img_path)).convert('RGB')
    img = self.transform(img)
    label = img_path.split('_')[-1].split('.')[0]
    label = re.sub(r'\([^)]*\)', '', label)
    label = int(label)
    return (img, label)

In [7]:
### Data Preprocessing ###

'''
# 데이터 정규화 추가
# 정규화 변환 정의
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
'''
transforms_train = transforms.Compose([transforms.Resize((448,448)),
                                       transforms.ToTensor()])
transforms_valtest = transforms.Compose([transforms.Resize((448,448)),
                                         transforms.ToTensor()])

BATCH_SIZE = 32
train_set = CUB2011(mode='train',
                    transform=transforms_train)
# transforms_train을 transforms_valtest로 변경
val_set = CUB2011(mode='valid',
                  transform=transforms_valtest)
test_set = CUB2011(mode='test',
                  transform=transforms_valtest)

print('Num of each dataset: ',len(train_set),len(val_set),len(test_set))

train_loader = DataLoader(train_set,batch_size=BATCH_SIZE,shuffle=True)
val_loader = DataLoader(val_set,batch_size=BATCH_SIZE,shuffle=False)
test_loader = DataLoader(test_set,batch_size=BATCH_SIZE,shuffle=False)

print("Loaded dataloader")

### Model / Optimizer ###
EPOCH = 25
lr = 0.0001

model = models.resnet18(pretrained=True)

### Tranfer Learning ###
num_features = model.fc.in_features
model.fc = nn.Linear(num_features,50)
model.to(DEVICE)

optimizer = AdamP(model.parameters(), lr=lr)

# 스케줄러 초기화 (get_cosine_schedule_with_warmup 사용)
total_steps = len(train_loader) * EPOCH
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=total_steps // 10, num_training_steps=total_steps)

print("Created a learning model and optimizer")

Num of each dataset:  2360 296 298
Loaded dataloader


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 68.6MB/s]


Created a learning model and optimizer


In [8]:
### Train/Evaluation ###
def train(model, train_loader, optimizer, epoch):
    model.train()
    train_loss_total = 0
    for i, (image, target) in enumerate(train_loader):
        image, target = image.to(DEVICE), target.to(DEVICE)
        output = model(image)
        optimizer.zero_grad()
        train_loss = F.cross_entropy(output, target).to(DEVICE)

        train_loss.backward()
        optimizer.step()

        train_loss_total += train_loss.item()

        if i % 10 == 0:
            print(f'Train Epoch: {epoch} [{i}/{len(train_loader)}]\tloss: {train_loss.item():.6f}')

    avg_train_loss = train_loss_total / len(train_loader)
    return avg_train_loss

def evaluate(model, val_loader):
    model.eval()
    eval_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (image, target) in enumerate(val_loader):
            image, target = image.to(DEVICE), target.to(DEVICE)
            output = model(image)

            eval_loss += F.cross_entropy(output, target, reduction='sum').item()
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()

    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)
    return eval_loss, eval_accuracy

In [9]:
### Main ###
start = time.time()
best = 0
for epoch in range(EPOCH):
    train_loss = train(model, train_loader, optimizer, epoch)
    val_loss, val_accuracy = evaluate(model, val_loader)

    # 스케줄러 업데이트 (에포크가 끝난 후)
    scheduler.step()

    # wandb 로그 기록
    wandb.log({
        'Epoch': epoch,
        'Train Loss': train_loss,
        'Validation Loss': val_loss,
        'Validation Accuracy': val_accuracy
    })

    # Save best model
    if val_accuracy > best:
        best = val_accuracy
        torch.save(model.state_dict(), "./best_model.pth")

    print(f"[{epoch}] Validation Loss: {val_loss:.4f}, Accuracy: {val_accuracy:.4f}%")

# Test result
test_loss, test_accuracy = evaluate(model, test_loader)
print(f'[FINAL] Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}%')

# wandb 테스트 결과 로그 기록
wandb.log({
    'Test Loss': test_loss,
    'Test Accuracy': test_accuracy
})

end = time.time()
elapsed_time = end - start

print("Best Accuracy: ", best)
print(f"Elapsed Time: {int(elapsed_time / 3600)}h, {int(elapsed_time / 60)}m, {int(elapsed_time % 60)}s")

wandb.finish()


Train Epoch: 0 [0/74]	loss: 3.995575
Train Epoch: 0 [10/74]	loss: 4.097239
Train Epoch: 0 [20/74]	loss: 4.014915
Train Epoch: 0 [30/74]	loss: 4.084760
Train Epoch: 0 [40/74]	loss: 3.996701
Train Epoch: 0 [50/74]	loss: 4.192347
Train Epoch: 0 [60/74]	loss: 4.165475
Train Epoch: 0 [70/74]	loss: 4.137008
[0] Validation Loss: 4.0482, Accuracy: 2.3649%
Train Epoch: 1 [0/74]	loss: 4.157443
Train Epoch: 1 [10/74]	loss: 4.093915
Train Epoch: 1 [20/74]	loss: 4.002744
Train Epoch: 1 [30/74]	loss: 4.083667
Train Epoch: 1 [40/74]	loss: 4.067531
Train Epoch: 1 [50/74]	loss: 4.148960
Train Epoch: 1 [60/74]	loss: 4.003203
Train Epoch: 1 [70/74]	loss: 4.105718
[1] Validation Loss: 4.0333, Accuracy: 2.3649%
Train Epoch: 2 [0/74]	loss: 3.918234
Train Epoch: 2 [10/74]	loss: 3.966890
Train Epoch: 2 [20/74]	loss: 4.042470
Train Epoch: 2 [30/74]	loss: 4.037005
Train Epoch: 2 [40/74]	loss: 4.053275
Train Epoch: 2 [50/74]	loss: 4.055900
Train Epoch: 2 [60/74]	loss: 4.014310
Train Epoch: 2 [70/74]	loss: 4.0989

Epoch,▁▁▂▂▂▂▃▃▃▄▄▄▅▅▅▅▆▆▆▇▇▇▇██
Test Accuracy,▁
Test Loss,▁
Train Loss,█████▇▇▇▇▆▆▆▅▅▄▄▄▃▃▃▂▂▂▁▁
Validation Accuracy,▁▁▁▁▁▁▂▂▂▂▃▃▄▄▅▅▆▆▇▇▇▇███
Validation Loss,█████▇▇▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▂▁▁
Epoch,24
Test Accuracy,78.52349
Test Loss,1.59237
Train Loss,1.18749
Validation Accuracy,76.01351
